In [1]:
import sys
import os
from scipy import stats
import pandas as pd
import numpy as np
import seaborn as sns
thisDir = os.path.expanduser('~/Desktop/MSC_Alexis/analysis/')
dataDir = thisDir + 'data/mvpa_data/'
outDir = thisDir + 'output/results/acc/'
taskList=['mem','motor','semantic','glass']

In [69]:
#Difference score CVmDS permutation
pST=pd.read_csv(thisDir+'output/results/permutation/CVmDS_acc.csv',na_values=[9999])


DS=pd.read_csv(outDir+'DS/acc.csv')
#CV
CV=pd.read_csv(outDir+'CV/acc.csv')
#permutation testing CV DS
DS.rename(columns={'train_sub':'sub'},inplace=True)
same_task=CV.merge(DS,how='left',on=['task','sub'],suffixes=('','_DS'))
#SS-OS
same_task['diff'] =same_task.acc-same_task.acc_DS 
#subset so we only average the SS-OS per train/test tasks
ST=same_task[['task','diff']]
#take average
diffST=ST.groupby(['task']).mean()
diffST.reset_index(inplace=True)

#prepping BS and SS
BS.rename(columns={'train_sub':'sub'},inplace=True)

diff_task=SS.merge(BS,how='left',on=['train_task','test_task','sub'],suffixes=('','_BS'))
#SS-OS
diff_task['diff'] =diff_task.acc-diff_task.acc_BS 
#subset so we only average the SS-OS per train/test tasks
DT=diff_task[['train_task','test_task','diff']]
#take average
diffDT=DT.groupby(['train_task','test_task']).mean()
diffDT.reset_index(inplace=True)
diffDT['TrTe'] = diffDT.train_task.map(str) + "_" + diffDT.test_task
DT=diffDT[['TrTe','diff']]
DT.set_index('TrTe',inplace=True)


#CVmDS
#dataframe of pvals for each task in same task 
#diffST.set_index('task',inplace=True)
tasks=[]
p_val=[]

for task in taskList:
    muST=diffST.loc[diffST['task']==task]
    muST=muST.iloc[0,1]
    ST_perm=pST.loc[pST['task']==task]
    ST_perm_keep=ST_perm[ST_perm['diff']>muST]
    count=ST_perm_keep['diff'].count()
    p=(count+1)/1001
    tasks.append(task)
    p_val.append(p)
ST_all=pd.DataFrame({'task':tasks, 'pval':p_val})
ST_all.set_index('task',inplace=True)
ST_all

,pval
task,
mem,0.000999
motor,0.000999
semantic,0.000999
glass,0.000999


In [41]:
#permutation testing SS BS
#permuted different task (SS-OS) SS-BS
pDT=pd.read_csv(thisDir+'output/results/permutation/SSmBS_acc.csv',na_values=[9999])
"""
SS=pd.read_csv(outDir+'SS/acc.csv')
BS=pd.read_csv(outDir+'BS/acc.csv')
SS.drop(columns=['spec','sen'],inplace=True)
SS=SS.groupby(['train_task','test_task']).mean()
SS.reset_index(inplace=True)
#subset so we only average train/test tasks

#take average
BS.drop(columns=['spec','sen'],inplace=True)
BS=BS.groupby(['train_task','test_task']).mean()
BS.reset_index(inplace=True)

diffTask=SS.merge(BS,how='left',on=['train_task','test_task'],suffixes=('','_BS'))
diffTask['diff']=diffTask['acc']-diffTask['acc_BS']
diffTask['TrTe'] = diffTask.train_task.map(str) + "_" + diffTask.test_task
DT=diffDT[['TrTe','diff']]
DT.set_index('TrTe',inplace=True)
#DT=diffTask[['train_task','test_task','diff']]
#DT.set_index('train_task',inplace=True)
#dataframe to hold p values for each task by task
"""
training=[]
testing=[]
p_values=[]
for train_task in taskList:
    for test_task in taskList:
        if train_task==test_task:
            continue
        else:
            DTmu=DT.loc[train_task+'_'+test_task].values
            DT_perm=pDT.loc[(pDT['train_task']==train_task) & (pDT['test_task']==test_task)]
            DT_perm_kept=DT_perm[DT_perm['diff']>DTmu[0]]
            count=DT_perm_kept['diff'].count()
            pval=(count+1)/(1001)
            training.append(train_task)
            testing.append(test_task)
            p_values.append(pval)
DT_all=pd.DataFrame({'train_task':training,'test_task':testing,'pval':p_values})
DT_all

,train_task,test_task,pval
0,mem,motor,0.043956
1,mem,semantic,0.038961
2,mem,glass,0.008991
3,motor,mem,0.014985
4,motor,semantic,0.034965
5,motor,glass,0.002997
6,semantic,mem,0.000999
7,semantic,motor,0.000999
8,semantic,glass,0.000999
9,glass,mem,0.000999


In [108]:
#permutation testing SS
###############################
###############################
#permuted labels SS
SS_permuted=pd.read_csv(thisDir+'output/results/permutation/SS/acc.csv',na_values=[9999])
#actual scores SS 
SS=pd.read_csv(outDir+'SS/acc.csv')
#subset so we only average train/test tasks
SS=SS[['train_task','test_task','acc']]
#take average
avg=SS.groupby(['train_task', 'test_task']).mean()
avg.reset_index(inplace=True)
avg['TrTe'] = avg.train_task.map(str) + "_" + avg.test_task
avg=avg[['TrTe','acc']]
avg.set_index('TrTe',inplace=True)

#dataframe to hold p values for each task by task
training=[]
testing=[]
p_values=[]
for train_task in taskList:
    for test_task in taskList:
        if train_task==test_task:
            continue
        else:
            Scmu=avg.loc[train_task+'_'+test_task].values
            SS_p=SS_permuted.loc[(SS_permuted['train_task']==train_task) & (SS_permuted['test_task']==test_task)]
            SS_p_kept=SS_p[SS_p['acc']>Scmu[0]]
            count=SS_p_kept['acc'].count()
            pval=(count+1)/(1001)
            training.append(train_task)
            testing.append(test_task)
            p_values.append(pval)
SS_p_all=pd.DataFrame({'train_task':training,'test_task':testing,'pval':p_values})
SS_p_all

,train_task,test_task,pval
0,mem,motor,0.002997
1,mem,semantic,0.008991
2,mem,glass,0.010989
3,motor,mem,0.092907
4,motor,semantic,0.095904
5,motor,glass,0.059940
6,semantic,mem,0.023976
7,semantic,motor,0.061938
8,semantic,glass,0.007992
9,glass,mem,0.011988


In [109]:
BS_permuted=pd.read_csv(thisDir+'output/results/permutation/BS/acc.csv',na_values=[9999])
#actual scores
BS=pd.read_csv(outDir+'BS/acc.csv')
#subset so we only average train/test tasks
BS=BS[['train_task','test_task','acc']]
#take average
avg=BS.groupby(['train_task', 'test_task']).mean()
avg.reset_index(inplace=True)
avg['TrTe'] = avg.train_task.map(str) + "_" + avg.test_task
avg=avg[['TrTe','acc']]
avg.set_index('TrTe',inplace=True)

#dataframe to hold p values for each task by task
training=[]
testing=[]
p_values=[]
for train_task in taskList:
    for test_task in taskList:
        if train_task==test_task:
            continue
        else:
            Scmu=avg.loc[train_task+'_'+test_task].values
            BS_p=BS_permuted.loc[(BS_permuted['train_task']==train_task) & (BS_permuted['test_task']==test_task)]
            BS_p_kept=BS_p[BS_p['acc']>Scmu[0]]
            count=BS_p_kept['acc'].count()
            pval=(count+1)/(1009)
            training.append(train_task)
            testing.append(test_task)
            p_values.append(pval)
BS_p_all=pd.DataFrame({'train_task':training,'test_task':testing,'pval':p_values})
BS_p_all

,train_task,test_task,pval
0,mem,motor,0.010902
1,mem,semantic,0.003964
2,mem,glass,0.014866
3,motor,mem,0.111001
4,motor,semantic,0.114965
5,motor,glass,0.138751
6,semantic,mem,0.198216
7,semantic,motor,0.248761
8,semantic,glass,0.210109
9,glass,mem,0.155600


In [115]:
CV_permuted=pd.read_csv(thisDir+'output/results/permutation/CV/acc.csv',na_values=[9999])
#actual scores
CV=pd.read_csv(outDir+'CV/acc.csv')
#subset so we only average train/test tasks
CV=CV[['task','acc']]
#take average
avg=CV.groupby(['task']).mean()
#dataframe to hold p values for each task by task
training=[]
p_values=[]
for train_task in taskList:
    Scmu=avg.loc[train_task].values
    CV_p=CV_permuted.loc[(CV_permuted['task']==train_task)]
    CV_p_kept=CV_p[CV_p['acc']>Scmu[0]]
    count=CV_p_kept['acc'].count()
    pval=(count+1)/(1001)
    training.append(train_task)
    testing.append(test_task)
    p_values.append(pval)
CV_p_all=pd.DataFrame({'train_task':training,'pval':p_values})
CV_p_all

,train_task,pval
0,mem,0.000999
1,motor,0.000999
2,semantic,0.000999
3,glass,0.000999


In [116]:
DS_permuted=pd.read_csv(thisDir+'output/results/permutation/DS/acc.csv',na_values=[9999])
#actual scores
DS=pd.read_csv(outDir+'DS/acc.csv')
#subset so we only average train/test tasks
DS=DS[['task','acc']]
#take average
avg=DS.groupby(['task']).mean()
#dataframe to hold p values for each task by task
training=[]
p_values=[]
for train_task in taskList:
    Scmu=avg.loc[train_task].values
    DS_p=DS_permuted.loc[(DS_permuted['task']==train_task)]
    DS_p_kept=DS_p[DS_p['acc']>Scmu[0]]
    count=DS_p_kept['acc'].count()
    pval=(count+1)/(1009)
    training.append(train_task)
    testing.append(test_task)
    p_values.append(pval)
DS_p_all=pd.DataFrame({'train_task':training,'pval':p_values})
DS_p_all

,train_task,pval
0,mem,0.003964
1,motor,0.006938
2,semantic,0.072349
3,glass,0.162537


In [12]:
#testing sensitivity CVmSS 
#this procedure combines performance of the testing task so the combined plot I can test for significance
#permutation 
CVmSS_permuted=pd.read_csv(thisDir+'output/results/permutation/STmDT_CVmSS_acc.csv',na_values=[9999])

#actual scores SS 
CV=pd.read_csv(outDir+'CV/acc.csv')
SS=pd.read_csv(outDir+'SS/acc.csv')
SS.drop(columns=['spec','sen'])
SS=SS.groupby(['train_task']).mean()
SS.reset_index(inplace=True)
#subset so we only average train/test tasks
CV=CV[['task','acc']]
#take average
avgCV=CV.groupby(['task']).mean()
avgCV.reset_index(inplace=True)
avgCV.rename(columns={'task':'train_task'},inplace=True)

diffTask=avg.merge(avgCV,how='left',on=['train_task'],suffixes=('','_CV'))
diffTask['diff']=diffTask['acc_CV']-diffTask['acc']
avg=diffTask[['train_task','diff']]
avg.set_index('train_task',inplace=True)


In [15]:
taskList=['mem','motor','semantic','glass']
#dataframe to hold p values for each task by task
training=[]
testing=[]
p_values=[]
for train_task in taskList:
    Scmu=avg.loc[train_task].values
    btw=CVmSS_permuted.loc[(CVmSS_permuted['train_task']==train_task)]
    btw_kept=btw[btw['diff']>Scmu[0]]
    count=btw_kept['diff'].count()
    pval=(count+1)/(3001) #since keeping all testing tasks included 
    training.append(train_task)
    p_values.append(pval)
btw_all=pd.DataFrame({'train_task':training,'pval':p_values})
btw_all

,train_task,pval
0,mem,0.034322
1,motor,0.000333
2,semantic,0.005332
3,glass,0.625125


In [33]:
#testing sensitivity CVmSS
#permutation 
CVmSS_permuted=pd.read_csv(thisDir+'output/results/permutation/STmDT_CVmSS_acc.csv',na_values=[9999])

#actual scores SS 
CV=pd.read_csv(outDir+'CV/acc.csv')
SS=pd.read_csv(outDir+'SS/acc.csv')
#subset so we only average train/test tasks
SS=SS[['train_task','test_task','acc']]
#take average
avg=SS.groupby(['train_task', 'test_task']).mean()
avg.reset_index(inplace=True)

#subset so we only average train/test tasks
CV=CV[['task','acc']]
#take average
avgCV=CV.groupby(['task']).mean()
avgCV.reset_index(inplace=True)
avgCV.rename(columns={'task':'train_task'},inplace=True)

diffTask=avg.merge(avgCV,how='left',on=['train_task'],suffixes=('','_CV'))
diffTask['diff']=diffTask['acc_CV']-diffTask['acc']
diffTask['TrTe'] = diffTask.train_task.map(str) + "_" + diffTask.test_task
avg=diffTask[['TrTe','diff']]
avg.set_index('TrTe',inplace=True)

taskList=['mem','motor','semantic','glass']
#dataframe to hold p values for each task by task
training=[]
testing=[]
p_values=[]
for train_task in taskList:
    for test_task in taskList:
        if train_task==test_task:
            continue
        else:
            Scmu=avg.loc[train_task+'_'+test_task].values
            btw=CVmSS_permuted.loc[(CVmSS_permuted['train_task']==train_task) & (CVmSS_permuted['test_task']==test_task)]
            btw_kept=btw[btw['diff']>Scmu[0]]
            count=btw_kept['diff'].count()
            pval=(count+1)/(1001)
            training.append(train_task)
            testing.append(test_task)
            p_values.append(pval)
btw_all=pd.DataFrame({'train_task':training,'test_task':testing,'pval':p_values})
btw_all
######################################################
#permutation test for CVmSS testing task specificity##
######################################################

,train_task,test_task,pval
0,mem,motor,0.034965
1,mem,semantic,0.066933
2,mem,glass,0.012987
3,motor,mem,0.000999
4,motor,semantic,0.000999
5,motor,glass,0.000999
6,semantic,mem,0.000999
7,semantic,motor,0.000999
8,semantic,glass,0.024975
9,glass,mem,0.181818


In [32]:
avg

,acc
TrTe,
glass_mem,0.830139
glass_motor,0.808611
glass_semantic,0.935625
mem_glass,0.878681
mem_motor,0.888125
mem_semantic,0.916875
motor_glass,0.738403
motor_mem,0.693750
motor_semantic,0.663125


In [26]:
CVmSS_permuted.head()

,train_task,test_task,diff
0,glass,mem,0.021458
1,glass,motor,0.022639
2,glass,semantic,0.028958
3,mem,glass,0.063472
4,mem,motor,-0.086875


In [72]:
#Calculating SD and Mean for all regular analysis
def statsACC(df,analysis):
    if analysis=='CV':
        print('cross validation stats')
        stats=df.groupby('task').mean()
        stats.rename(columns={'acc':'Mean'},inplace=True)
        sd=df.groupby('task').std()
        stats['Std']=sd['acc']
        stats.to_csv(outDir+analysis+'/stats.csv', index=True)
    elif analysis=='SS':
        print('same sub stats')
        df.drop(['sub'], axis=1, inplace=True)
        stats=df.groupby(['test_task', 'train_task']).mean()
        stats.rename(columns={'acc':'Mean'}, inplace=True)
        sd=df.groupby(['test_task', 'train_task']).std()
        stats['Std']=sd['acc']
        stats.reset_index(inplace=True)
        stats.to_csv(outDir+analysis+'/stats.csv', index=True)
    elif analysis=='DS':
        print('diff sub stats')
        df.drop(['train_sub', 'test_sub'], axis=1, inplace=True)
        stats=df.groupby('task').mean()
        stats.rename(columns={'acc':'Mean'}, inplace=True)
        sd=df.groupby('task').std()
        stats['Std']=sd['acc']
        stats.to_csv(outDir+analysis+'/stats.csv', index=True)
    elif analysis=='BS':
        print('diff sub diff task stats')
        df.drop(['train_sub', 'test_sub'], axis=1, inplace=True)
        stats=df.groupby(['test_task', 'train_task']).mean()
        stats.rename(columns={'acc':'Mean'}, inplace=True)
        sd=df.groupby(['test_task', 'train_task']).std()
        stats['Std']=sd['acc']
        stats.reset_index(inplace=True)
        stats.to_csv(outDir+analysis+'/stats.csv', index=True)
    return stats

In [79]:
#statsACC(CV,'CV')
statsACC(SS,'SS')
#statsACC(DS,'DS')
#statsACC(BS,'BS')


same sub stats


,test_task,train_task,Mean,Std
0,glass,mem,0.841806,0.141014
1,glass,motor,0.658750,0.148246
2,glass,semantic,0.807083,0.101249
3,mem,glass,0.760694,0.139531
4,mem,motor,0.632500,0.103957
5,mem,semantic,0.694375,0.123446
6,motor,glass,0.792569,0.142429
7,motor,mem,0.876250,0.120971
8,motor,semantic,0.628750,0.093417
9,semantic,glass,0.900069,0.073824


In [82]:
ALL['cv_acc'].mean()
ALL['cv_acc'].std()

0.010808903106404302

In [84]:
#ALL['acc'].mean()
ALL['acc'].std()

0.11435408419187446

In [26]:
SS=pd.read_csv(outDir+'SS/acc.csv')

In [30]:
SS.groupby(['train_task']).mean()

,acc,spec,sen
train_task,,,
glass,0.858125,0.986250,0.730000
mem,0.894560,0.987731,0.801389
motor,0.698426,0.999583,0.397269
semantic,0.773403,0.999583,0.547222


In [22]:
SS.groupby(['train_task','test_task']).mean()

acc      spec       sen
train_task test_task                              
glass      mem        0.830139  0.986250  0.674028
           motor      0.808611  0.973750  0.643472
           semantic   0.935625  0.998750  0.872500
mem        glass      0.878681  0.984444  0.772917
           motor      0.888125  0.983750  0.792500
           semantic   0.916875  0.995000  0.838750
motor      glass      0.738403  1.000000  0.476806
           mem        0.693750  0.998750  0.388750
           semantic   0.663125  1.000000  0.326250
semantic   glass      0.844583  1.000000  0.689167
           mem        0.772500  0.998750  0.546250
           motor      0.703125  1.000000  0.406250

In [23]:
SS.groupby(['train_task','test_task']).std()

acc      spec       sen
train_task test_task                              
glass      mem        0.114660  0.038891  0.206827
           motor      0.153236  0.066534  0.262088
           semantic   0.051508  0.003536  0.102365
mem        glass      0.108183  0.016211  0.210060
           motor      0.086104  0.034200  0.176857
           semantic   0.080664  0.007559  0.164007
motor      glass      0.164101  0.000000  0.328202
           mem        0.156610  0.003536  0.312796
           semantic   0.142326  0.000000  0.284652
semantic   glass      0.097304  0.000000  0.194608
           mem        0.143203  0.003536  0.286752
           motor      0.120947  0.000000  0.241894

In [8]:
CV.groupby(['task']).mean()

,acc,spec,sen
task,,,
glass,0.878472,1.0,0.756944
mem,0.993750,1.0,0.987500
motor,0.981250,1.0,0.962500
semantic,0.943750,1.0,0.887500


In [9]:
CV.groupby(['task']).std()

,acc,spec,sen
task,,,
glass,0.064065,0.0,0.128131
mem,0.017678,0.0,0.035355
motor,0.025877,0.0,0.051755
semantic,0.032043,0.0,0.064087


In [11]:
DS.groupby(['task']).mean()

,acc,spec,sen
task,,,
glass,0.577480,0.996071,0.158889
mem,0.894375,0.821964,0.966786
motor,0.788393,0.843750,0.733036
semantic,0.669911,0.998214,0.341607


In [12]:
DS.groupby(['task']).std()

,acc,spec,sen
task,,,
glass,0.101508,0.014607,0.208554
mem,0.100476,0.210743,0.067017
motor,0.154729,0.282261,0.311407
semantic,0.144881,0.005430,0.291627


In [14]:
BS.groupby(['train_task','test_task']).mean()

acc      spec       sen
train_task test_task                              
glass      mem        0.571042  0.996071  0.146012
           motor      0.584782  0.996071  0.173492
           semantic   0.577837  0.996071  0.159603
mem        glass      0.764921  0.821726  0.708115
           motor      0.821071  0.821964  0.820179
           semantic   0.790804  0.821964  0.759643
motor      glass      0.605476  0.843770  0.367183
           mem        0.620179  0.843750  0.396607
           semantic   0.616518  0.843750  0.389286
semantic   glass      0.555040  0.998214  0.111865
           mem        0.562411  0.998214  0.126607
           motor      0.553036  0.998214  0.107857

In [16]:
BS.groupby(['train_task','test_task']).std()

acc      spec       sen
train_task test_task                              
glass      mem        0.114925  0.014607  0.235733
           motor      0.102336  0.014607  0.212573
           semantic   0.094114  0.014607  0.191819
mem        glass      0.131100  0.210683  0.243705
           motor      0.121392  0.210743  0.197581
           semantic   0.118036  0.210743  0.192986
motor      glass      0.098486  0.282237  0.338109
           mem        0.114638  0.282261  0.376767
           semantic   0.116396  0.282261  0.356059
semantic   glass      0.087032  0.005430  0.175022
           mem        0.095594  0.005430  0.192347
           motor      0.085810  0.005430  0.171620

In [32]:
import pandas as pd

rest=pd.read_csv('~/Desktop/rest.csv')

rest['min']=(rest['rest']*2.2)/60

#rest=rest[rest['min']>30]

In [33]:
rest['half']=rest['min']/2
rest['fourth']=rest['min']/4

In [34]:
rest

,sub,rest,min,half,fourth
0,MSC01,523,19.176667,9.588333,4.794167
1,MSC01,717,26.290000,13.145000,6.572500
2,MSC01,732,26.840000,13.420000,6.710000
3,MSC01,778,28.526667,14.263333,7.131667
4,MSC01,710,26.033333,13.016667,6.508333
...,...,...,...,...,...
75,MSC10,336,12.320000,6.160000,3.080000
76,MSC10,545,19.983333,9.991667,4.995833
77,MSC10,526,19.286667,9.643333,4.821667
78,MSC10,766,28.086667,14.043333,7.021667


In [2]:
#permutation comparing ALL performance with permuted ALL

ALL=pd.read_csv(outDir+'ALL/shufflekFold_acc.csv')
ALL_perm=pd.read_csv(thisDir+'output/results/permutation/ALL_acc.csv',na_values=[9999])
ALL_perm.head()
ALL['diff']=ALL['cv_acc']-ALL['acc']
ALL_avg=ALL['diff'].mean()

,diff_acc,SS,OS
0,0.005497,0.509216,0.503718
1,-0.000797,0.500670,0.501467
2,0.024668,0.511830,0.487162
3,-0.005880,0.517889,0.523769
4,0.012207,0.505772,0.493565


In [3]:
perm_kept=ALL_perm[ALL_perm['diff_acc']>ALL_avg]
count=perm_kept['diff_acc'].count()
pval=(count+1)/(1001)

In [4]:
pval

0.000999000999000999

In [5]:
ALL_cv_avg=ALL['cv_acc'].mean()
ALL_ds_avg=ALL['acc'].mean()

In [6]:
perm_kept=ALL_perm[ALL_perm['SS']>ALL_cv_avg]
count=perm_kept['SS'].count()
pval=(count+1)/(1001)

In [7]:
pval

0.000999000999000999

In [8]:
perm_kept=ALL_perm[ALL_perm['OS']>ALL_ds_avg]
count=perm_kept['OS'].count()
pval=(count+1)/(1001)

In [9]:
pval

0.000999000999000999

In [73]:
sixty_res.head()

,Days,Analysis,acc
0,30,Same Subject,0.966667
1,30,Same Subject,0.966667
2,30,Same Subject,0.966667
3,30,Same Subject,0.966667
4,30,Same Subject,0.966667


In [74]:
sixty_res=sixty_res[sixty_res['Analysis']=='Same Subject']

In [78]:
five=sixty_res[sixty_res['Days']==5]

ten=sixty_res[sixty_res['Days']==10]

thirty=sixty_res[sixty_res['Days']==30]


In [80]:
five['acc'].mean()

0.6737499999999973

In [81]:
ten['acc'].mean()

0.8634375000000022

In [82]:
thirty['acc'].mean()

0.9831250000000048

In [94]:
#BS['acc'].mean()
BS['acc'].std()

0.1420709585872698

In [95]:
SS['acc'].mean()
SS['acc'].std()

0.14540192001446242

In [23]:
#permutation testing SS BS for condensed plot
#permuted different task (SS-OS) SS-BS
pDT=pd.read_csv(thisDir+'output/results/permutation/SSmBS_acc.csv',na_values=[9999])
#prepping BS and SS


#actual scores SS 
SS=pd.read_csv(outDir+'SS/acc.csv')
BS=pd.read_csv(outDir+'BS/acc.csv')
SS.drop(columns=['spec','sen'],inplace=True)
SS=SS.groupby(['train_task']).mean()
SS.reset_index(inplace=True)
#subset so we only average train/test tasks

#take average
BS.drop(columns=['spec','sen'],inplace=True)
BS=BS.groupby(['train_task']).mean()
BS.reset_index(inplace=True)

diffTask=SS.merge(BS,how='left',on=['train_task'],suffixes=('','_BS'))


diffTask['diff']=diffTask['acc']-diffTask['acc_BS']
avg=diffTask[['train_task','diff']]
avg.set_index('train_task',inplace=True)

#dataframe to hold p values for each task by task
training=[]
p_values=[]
for train_task in taskList:
    DTmu=avg.loc[train_task].values
    DT_perm=pDT.loc[(pDT['train_task']==train_task)]
    DT_perm_kept=DT_perm[DT_perm['diff']>DTmu[0]]
    count=DT_perm_kept['diff'].count()
    pval=(count+1)/(3001)#since keeping all 3 testing tasks
    training.append(train_task)
    testing.append(test_task)
    p_values.append(pval)
DT_all=pd.DataFrame({'train_task':training,'pval':p_values})
DT_all

,train_task,pval
0,mem,0.026658
1,motor,0.010663
2,semantic,0.000333
3,glass,0.000333


In [19]:
DT_perm=pDT.loc[(pDT['train_task']=='mem')]

In [55]:
ALL.cv_acc.mean()-ALL.acc.mean()

0.24929209183673473

In [62]:
CV[CV['task']=='glass'].acc.mean()-DS[DS['task']=='glass'].acc.mean()

0.30099206349206364

In [63]:
CV[CV['task']=='semantic'].acc.mean()-DS[DS['task']=='semantic'].acc.mean()

0.27383928571428584

In [64]:
CV[CV['task']=='motor'].acc.mean()-DS[DS['task']=='motor'].acc.mean()

0.1562819664902999

In [65]:
CV[CV['task']=='mem'].acc.mean()-DS[DS['task']=='mem'].acc.mean()

0.09727678571428611

In [67]:
DS.groupby('task').mean()

,acc,spec,sen
task,,,
glass,0.577480,0.996071,0.158889
mem,0.888661,0.868929,0.908393
motor,0.824968,0.991270,0.658666
semantic,0.669911,0.998214,0.341607


In [106]:
DS.loc[(DS['task']=='mem')].shape

(56, 2)